# Compare Cre Lines

This notebook will allow you to dive into the Allen Institute for Brain Science SDK in order to test the electrophysiological differences between different genetically-identified cell types. 

### Tutorial Steps
1. [Setup our coding environment](#setup)
2. [Get the Data for Different Cre Lines](#credata)
3. [Plot Pre-Computed Features for Different Cre Lines](#plot)
<hr>

<a id="setup"></a>
## Step 1. Setup our coding environment
Whenever we start an analysis in Python, we need to be sure to import the necessary code packages. Below, we'll import a common selection of packages that will help us analyze and plot our data. We'll also configure the plotting in our notebook.

><b>Task</b>: First, we need to make sure that our coding environment has Numpy, Pandas, and Matplotlib already installed. Run the cell below -- any packages that are missing will be installed for you.

In [ ]:
# This will ensure that Numpy, Pandas, and Matplotlib are installed.
try:
    import numpy
    print('numpy already installed')
except ImportError as e:
    !pip install numpy
try:
    import pandas
    print('pandas already installed')
except ImportError as e:
    !pip install pandas  
try:
    import matplotlib
    print('matplotlib already installed')
except ImportError as e:
    !pip install matplotlib

><b>Task</b>: Run the cell below.

In [ ]:
import matplotlib.pyplot as plt # Import our plotting package from matplotlib
%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
import pandas as pd # Import pandas for working with databases
import numpy as np
print('Packages imported!')

The Allen Institute has compiled a set of code and tools called a **Software Development Kit** (SDK). We need to make sure that you have this installed in your environment.

See [Technical Notes](#technical) at the end of this notebook for more information about working with the AllenSDK.

><b>Task</b>: Run the cell below. If you receive an error, there are additional instructions on how to install the SDK locally <a href="http://alleninstitute.github.io/AllenSDK/install.html">here</a>.

In [ ]:
# This will ensure that the AllenSDK is installed.
try:
    import allensdk
    if allensdk.__version__ == '2.2.0':
        print('allensdk version ' + allensdk.__version__ + ' already installed')
    else:
        print('allensdk installed with an older version. some features may not work.')
except ImportError as e:
    !pip install allensdk

Now that we have the allensdk installed, we can `import` the CellTypesCache module. This module provides tools to allow us to get information from the Cell Types database. 

The CellTypesCache that we're importing provides tools to allow us to get information from the cell types database. We're giving it a **manifest** filename as well. CellTypesCache will create this manifest file, which contains metadata about the cache. If you want, you can look in the cell_types folder in your code directory and take a look at the file.

><b>Task</b>: Run the cell below.

In [ ]:
#Import the "Cell Types Cache" from the AllenSDK core package
from allensdk.core.cell_types_cache import CellTypesCache

#Initialize the cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json')

print('CellTypesCache imported.')

<a id="credata"></a>

## Step 2. Get the data for different Cre transgenic mouse lines
What if we want to know whether different genetically-identified cells have different intrinsic physiology?
    
The Allen Institute for Brain Science uses transgenic mouse lines that have Cre-expressing cells to mark specific types of cells in the brain. This technology is called the Cre-Lox system, and is a common way in neuroscience (and some other fields) to target cells based on their expression of specific genetic promotors. For more information about Cre/Lox technology, see [this website](https://old.abmgood.com/marketing/knowledge_base/Cre-Lox_Recombination.php).

Let's first find out which Cre lines are available in our dataset.

>**Task**: Run the cell below. It will print a list of Cre lines available in this dataset.

In [ ]:
from allensdk.api.queries.cell_types_api import CellTypesApi

# make a dataframe out of ephys features
ephys_features = ctc.get_ephys_features()
ephys_features_df = pd.DataFrame.from_records(ephys_features)

# grab mouse data and merge with dataframe
mouse_df = pd.DataFrame(ctc.get_cells(species=[CellTypesApi.MOUSE])) # get all mouse experiments
mouse_ephys_df = pd.merge(mouse_df,ephys_features_df,left_on='id',right_on='specimen_id',how='left')

print(mouse_ephys_df['transgenic_line'].unique())

> **Task**: Choose two different Cre lines to compare, and assign them to the variables below by replacing the `...`. The value of your variable needs to be a <b>string</b> -- in other words, it should have quotes around it. The cell will print how many cells are in your datasets. If you have less than 10 cells, consider choosing a different Cre-line.

<div class="alert alert-block alert-info"><b>You can find information on the different cre-lines that are available in <a href="https://docs.google.com/document/d/1ZMMZgc7cS5BHhoWNqzjw95BdxOuj5wrYl9I7PV2HeUI/edit?usp=sharing">this glossary</a> or on the <a href="http://connectivity.brain-map.org/transgenic">Allen Institute's website</b></a>.</div>

> **Note**: Be sure that you are using the *entire* name of the Cre line -- that means *everything* within the single quotes above.

In [ ]:
cre_line_1 = ...
cre_line_2 = ...

cre_line_1_df = mouse_ephys_df[mouse_ephys_df['transgenic_line']==cre_line_1]
cre_line_2_df = mouse_ephys_df[mouse_ephys_df['transgenic_line']==cre_line_2]

print(cre_line_1 + ' has ' + str(len(cre_line_1_df)) + ' cells')
print(cre_line_2 + ' has ' + str(len(cre_line_2_df)) + ' cells')

<a id="plot"></a>
## Step 3. Plot Pre-Computed Features for Different Cre Lines
Let's start by plotting a distribution of the recorded resting membrane potential (`vrest`) for our two different Cre lines.

Note that the distribution here is normalized by the total count (`density=True`), since there may be different cell counts for your Cre lines. Look through the [`plt.hist()` documentation](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.pyplot.hist.html) for more information.

In [ ]:
plt.figure(figsize=(5,5)) # Modify this line to change the dimensions of your plot

# Change your parameter below.
# Make sure this matches the name in the parentheses in the orange box below.
parameter = 'vrest'

# Plot the histogram, with density = True 
plt.hist([cre_line_1_df[parameter],cre_line_2_df[parameter]],density = True)
plt.xlabel('Resting Membrane Potential (mV)') # X label -- be sure to update!
plt.ylabel('Number of Cells')
plt.legend([cre_line_1,cre_line_2])

# Boxplot creation lines below, we'll also drop NaN values
#plt.boxplot([cre_line_1_df[parameter].dropna(),cre_line_2_df[parameter].dropna()])
#plt.ylabel('This is the y label') # y-axis label
#plt.xticks([1, 2], [cre_line_1,cre_line_2])

plt.title('This is my plot title') # Plot title -- be sure to update!
plt.show()

> <b>Task</b>: Choose a different parameter to compare between your two different Cre lines, and rerun the plot above.
1. Use the documentation below to get the exact name of the parameter (in parentheses)
2. Set `parameter = ` to the exact name of your parameter. 
3. Decide whether or not you'd like to plot your data as a histogram or a boxplot. To make it a boxplot, <b>comment</b> out the lines for the histogram (add a <code>#</code>) and <b><i>uncomment</b></i> the four lines below. Re-run the code to get a boxplot of the data. 
4. Adjust the plot axis labels accordingly. You can also adjust the figure dimensions by changing `figsize=` in the first line.
5. Right click to save your image when you're done -- you'll need this for your data slide.

Below are a few metrics you might consider comparing. You can also find a complete glossary [here](https://docs.google.com/document/d/1YGLwkMTebwrXd_1E817LFbztMjSTCWh83Mlp3_3ZMEo/edit?usp=sharing):

<br>
<div style="background: #F3D48D; border-radius: 3px; padding: 10px;">
<b>Tau (<code>tau</code>)</b>: time constant of the membrane in milliseconds
    
<b>Adapation ratio (<code>adaptation</code>)</b>: The rate at which firing speeds up or slows down during a stimulus<br>
    
<b>Average ISI (<code>avg_isi</code>)</b>: The mean value of all interspike intervals in a sweep<br>

<b>Slope of f/I curve (<code>f_i_curve_slope</code>)</b>: slope of the curve between firing rate (f) and current injected<br>

<b>Input Resistance (<code>input_resistance_mohm</code>)</b>: The input resistance of the cell, in megaohms.<br>
    
<b>Voltage of after-hyperpolarization (<code>trough_v_short_square</code>)</b>: minimum value of the membrane potential during the after-hyperpolarization

<b>Action potential fast trough (<code>fast_trough_v_long_square</code>)</b>: Minimum value of the membrane potential in the interval lasting 5 ms after the peak.

<b>Upstroke/downstroke ratio (<code>upstroke_downstroke_ratio_long_square</code>)</b>: The ratio between the absolute values of the action potential peak upstroke and the action potential peak downstroke. </div>

Now that you have your two populations, let's see if they're statistically different. Below, we'll assign the data for each of your cre lines to two different variables, `cre_data_1` and `cre_data_2`.

>**Task:** Add code below to run two sample statistics on your data.

In [ ]:
from scipy import stats

cre_data_1 = cre_line_1_df[parameter].dropna() # Data for your first cre line
cre_data_2 = cre_line_2_df[parameter].dropna() # Data for your second cre line

# Add your statistics code here

# Test for normality

# Test for significant differences


That's it for today -- great work!

In [ ]:
from IPython.display import HTML
print('Nice work!')
HTML('<img src="https://media.giphy.com/media/xUOwGhOrYP0jP6iAy4/giphy.gif">')

<hr>
<a id="technical"></a>

### Technical notes & credits

This notebook demonstrates most of the features of the AllenSDK that help manipulate data in the Cell Types Database.  The main entry point will be through the `CellTypesCache` class. `CellTypesCache` is responsible for downloading Cell Types Database data to a standard directory structure on your hard drive.  If you use this class, you will not have to keep track of where your data lives, other than a root directory.

Much more information can be found in the <a href="http://help.brain-map.org/download/attachments/8323525/CellTypes_Ephys_Overview.pdf">Allen Brain Atlas whitepaper</a> as well as in their <a href="http://alleninstitute.github.io/AllenSDK/cell_types.html">GitHub documentation</a>.

This notebook was largely inspired by <a href='https://alleninstitute.github.io/AllenSDK/_static/examples/nb/cell_types.html'>this</a> notebook.